In [104]:
import pandas as pd
pd.set_option('display.max_rows', 500)

In [2]:
df = pd.read_csv('data.csv')

# 1. Общий анализ таблицы

In [3]:
df.shape

(1000000, 13)

In [4]:
df.head(5)

,datetime,user_ip,user_id,user_country_code,user_device_type,user_device_language,user_device_os,site_domain,site_page,site_ref,floor,price,is_win
0,2020-11-30 18:41:48,37.162.29.144,ka2rXU5cIb7c,ITA,4,it,Android,napoli.zon.it,https://napoli.zon.it/eredita-maradona-donero-...,http://m.facebook.com/,0.181539,0.272309,1
1,2020-11-30 18:41:48,37.163.56.102,jbf1Tx5J30Fb,ITA,4,it,Android,billoccino.com,https://www.billoccino.com/bill-gates-dopo-il-...,http://m.facebook.com/,0.352577,0.423092,1
2,2020-11-30 18:41:48,151.68.60.46,j87hqWFNPouc,ITA,4,it,Android,freedsound.net,https://freedsound.net/it/,https://www.google.com/,0.126633,0.189949,1
3,2020-11-30 18:41:48,95.249.118.85,k2oJLHERRX5c,ITA,4,it,Android,boardingmoms.com,https://boardingmoms.com/disappearing-into-fat...,http://m.facebook.com/,0.032824,0.039389,1
4,2020-11-30 18:41:48,79.42.244.127,kauBcU8n8IFa,ITA,4,it,Android,filosofiaspicciola.it,https://www.filosofiaspicciola.it/segni-zodiac...,http://m.facebook.com/,0.068230,0.075053,0


In [5]:
df.isna().sum()

datetime                    0
user_ip                     0
user_id                 73007
user_country_code           0
user_device_type            0
user_device_language        0
user_device_os              0
site_domain                 0
site_page               48134
site_ref                92163
floor                       0
price                       0
is_win                      0
dtype: int64

В таблице 1 000 000 значений, часть столбцов имеют пустые значения.

# 2. Поле datetime

In [6]:
df['datetime'] = pd.to_datetime(df['datetime'])

In [7]:
df['datetime'].dt.day.value_counts()

30    1000000
Name: datetime, dtype: int64

In [8]:
df['datetime'].dt.hour.value_counts()

19    88866
16    80904
12    77327
14    76368
15    73423
21    73383
18    60173
17    52454
11    47931
20    46364
22    45694
10    44950
7     39934
8     39212
9     38101
6     29115
0     21607
5     16777
13    15947
1     10320
4      8479
2      6625
3      6046
Name: datetime, dtype: int64

Поскольку все данные собраны в один и тот же день, всё кроме часа, минуты и секунды можно отбросить. Часы, минуты и секунды можно записать в три новых столбца либо в один стобец "часы", добавив к нему минуты и секуды как доли от часа

# 3. Поля user_ip и user_id

In [64]:
df[df['user_ip'].isin(df.groupby(['user_ip']).size().sort_values(ascending=False).head(50).index)].groupby(['user_ip','user_id']).size().head(50)

user_ip          user_id     
109.117.199.128  k8oT54xMNRXm    108
109.118.82.181   jb3Xh3MV2bTe    103
                 k4jiaVw4w6t9      3
                 k8hLStpisFH2      1
146.241.160.91   kau4ALdIuX7b      5
                 kauaNUrzY425      1
                 kaukPnRBxE25      2
                 kauvENWV3607    137
151.32.222.43    kad6voDi2GMg    114
151.36.47.162    i6er7sE8JCic    155
151.38.87.176    i6er7sE8JCic    242
                 k35XsG5A2qIb      1
                 k86PKOdxaae6      1
151.49.183.205   katTG4zLT13a    154
151.56.56.69     k7koWUsEGVqe    175
151.57.143.244   k93HqBSnxU1d    102
151.57.158.172   k93HqBSnxU1d    135
151.57.223.124   k93HqBSnxU1d    263
151.62.41.30     k12KA4oD52m0    123
                 k6ghj8m7Tspk     11
                 kauo7nVeQWK5     29
151.68.248.151   k8pf4Y11HrI4    148
                 katMYgN3_hBn     24
                 katdItoAjnon     16
151.74.154.19    ib5qqSQBRKP9    143
176.201.84.103   kahhtON0_hce    107
176.206.

In [67]:
df['user_id'].value_counts().shape[0]/df['user_ip'].value_counts().shape[0]

1.0461394798505905

Столбцы user_ip и user_id по большей части дублируют друг друга, но на каждый user_ip может приходиться несколько user_id. Поскольку столбец user_ip не может однозначно идентифицировать пользователя, он может быть отброшен.

Незначительная часть значений user_id отсутствуют (незарегистрированные пользователи?)

# 4. Поля user_country_code и user_device_language

In [68]:
df["user_country_code"].value_counts()

ITA    1000000
Name: user_country_code, dtype: int64

In [19]:
df['user_device_language'].value_counts()

it    1000000
Name: user_device_language, dtype: int64

Поля user_country_code и user_device_language в представленных данных имеют только одно значение. Стоит либо отбросить эти столбцы (поскольку они не могут улучшить качество модели) либо использовать другую выборку данных, в которых будут несколько значений этоих столбцов.

# 5. Поля user_device_type и user_device_os

In [24]:
df["user_device_os"].value_counts()

Android             612639
iOS                 174341
Windows OS          141862
Mac OS               32570
android              21703
windows               6861
ios                   4318
Other Desktop OS      2804
Other Tablet OS       1780
mac os                1008
Other Mobile OS         80
linux                   32
windows phone            2
Name: user_device_os, dtype: int64

In [71]:
df["user_device_type"].value_counts()

4    801874
2    185173
5     12283
1       669
3         1
Name: user_device_type, dtype: int64

In [ ]:
df["user_device_type"].value_counts()

In [75]:
df.groupby(['user_device_os','user_device_type']).size()

user_device_os    user_device_type
Android           4                   606052
                  5                     6587
Mac OS            2                    32570
Other Desktop OS  2                     2804
Other Mobile OS   4                       80
Other Tablet OS   5                     1780
Windows OS        2                   141862
android           1                      666
                  2                       38
                  4                    20261
                  5                      738
iOS               4                   171512
                  5                     2829
ios               1                        3
                  4                     3966
                  5                      349
linux             2                       31
                  3                        1
mac os            2                     1007
                  4                        1
windows           2                     6861
windows phone     4 

Ни одно из полей не содержит избыточную информацию и в предобработке не нуждается

# 6. Поля site_domain, site_page и site_ref

In [84]:
# df.groupby(['site_domain','site_page']).size().head(50)
df.groupby(['site_domain']).size().sort_values(ascending=True).head(50)

site_domain
01dd5ed4e9912cc9ef8e1d33ff91d99b.safeframe.googlesyndication.com    1
ergennaro.itput.info                                                1
lesyeuxorange.ruplayers.com                                         1
seoulmafia.itget.info                                               1
scherzidicoppia.itburn.info                                         1
emporiogenova.usmains.com                                           1
emamotorsport.ithomes.info                                          1
emamotorsport.itburn.info                                           1
liricalove.ruplayers.com                                            1
lorenzo1999ist.ithomes.info                                         1
lorettagracetv.itget.info                                           1
edupalla.ruplayers.com                                              1
edoardojannone.itget.info                                           1
ltposts.info                                                        1
lucasalv

In [142]:
# df[~df.site_domain.str.contains('^.*safeframe\.googlesyndication\.com')].groupby(['site_domain']).size().sort_values(ascending=True).head(50)
df.groupby(['site_domain']).size().sort_values(ascending=False).head(500).tail(20)

site_domain
miglioriaforismifrasibelle.usmains.com    5
ua-cam.com                                5
benedetta1972.usmains.com                 5
xfactoritalia.usmains.com                 5
www-mtlnovel-com.cdn.ampproject.org       5
ww.tvfun.video                            5
santatelevision.ruplayers.com             5
eleonorapetrella.itload.info              5
socialondon.com                           5
enimerwsi.eu                              5
ifattinews.it                             5
dirittoeconomia.net                       5
i2bomber.itmores.info                     4
lemiescienze.net                          4
smibiechannel.usmains.com                 4
newscatania.com                           4
hrvid.com                                 4
saporitotv.itburn.info                    4
gonfialarete.com                          4
us7.proxysite.one                         4
dtype: int64

In [147]:
df.groupby(['site_page']).size().sort_values(ascending=False).head(50)

site_page
https://notiziemaledette.it/news/dpcm-3-dicembre-tamponi-per-vedere-i-parenti-e-divieto-di-spostamenti/                                                                                    52444
https://freedsound.net/it/                                                                                                                                                                 35694
https://f1ingenerale.com/                                                                                                                                                                  30536
https://notiziemaledette.it/news/minuto-di-silenzio-per-maradona-calciatrice-si-rifiuta-di-rispettarlo-era-un-violentatore/                                                                21557
https://www.piudonna.it/signorini-nella-bufera-beccate-le-scene-scabrose-interviene-la-vigilanza-hanno-censurato-ma-si-e-visto-tutto-il-gf-va-chiuso-cose-successo/                        16086
https://www.comerisparmio

In [148]:
df.groupby(['site_ref']).size().sort_values(ascending=False).head(50)

site_ref
http://m.facebook.com/                                                                                                                                                                                      327313
http://m.facebook.com                                                                                                                                                                                        84129
https://www.google.com/                                                                                                                                                                                      71307
https://f1ingenerale.com/                                                                                                                                                                                    22935
https://pinterest.com/                                                                                                                             

В поле site_domain есть записи вида f4479ea832c23e94607abfaaa358702a.safeframe.googlesyndication.com или
lapizzafattaincasa.ruplayers.com. Они не отражают конкретный домен, но их количество незначительно и ими можно пренебречь.

В поле site_ref есть незначительно различающиеся записи, относящиеся к одной и той же ссылке (например http://m.facebook.com/ и http://m.facebook.com)
Их можно привести к одному виду
